In [ ]:
import os
import time
import torch
import torchvision
import numpy as np
from PIL import Image
from tempfile import TemporaryDirectory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/hymenoptera_data.zip -d /content/data

In [ ]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transform = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean, std)
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean, std)
    ]),
}

# ***writing our own custom class***

In [ ]:
class CustomData():
  def __init__(self, split, root_dir, transform):
    self.split = split
    self.root_dir = root_dir
    self.transform = transform

    self.image_path = []
    self.image_label = []

    self.class_map = {
        'ants' : 0,
        'bees' : 1
    }

    self.split_dir = os.path.join(self.root_dir, self.split)
    self.load_data()

  def load_data(self):
    for class_name in os.listdir(self.split_dir):
      class_dir = os.path.join(self.split_dir, class_name)
      if os.path.isdir(class_dir):
        for images in os.listdir(class_dir):
          image_dir = os.path.join(class_dir, images)
          if images.lower().endswith(('.jpeg', '.png', '.jpg')):
            self.image_path.append(image_dir)
            self.image_label.append(self.class_map[class_name])

  def __len__(self):
    return len(self.image_path)

  def __getitem__(self, index):
    img_path = self.image_path[index]
    image = Image.open(img_path).convert('RGB')

    if self.transform:
      image = self.transform(image)

    label = self.image_label[index]
    return image, label

In [ ]:
train_dataset = CustomData(
    root_dir = '/content/data/hymenoptera_data',
    transform = data_transform['train'],
    split = 'train'
)

val_dataset = CustomData(
    root_dir = '/content/data/hymenoptera_data',
    transform = data_transform['val'],
    split = 'val'
)

In [ ]:
batch_size = 4
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    dataset = val_dataset,
    batch_size = batch_size,
    shuffle = True
)


In [ ]:
# for input, output in train_loader:
#   print('input : ', input.shape)
#   print('output : ', output)
#   break

# print(len(train_dataset))
# print(len(val_dataset))

In [ ]:
dataloaders = {
    'train' : train_loader,
    'val' : val_loader
}

dataset_sizes = {
    'train' : len(train_dataset),
    'val' : len(val_dataset)
}

In [ ]:
print(dataset_sizes)
print(dataloaders)

{'train': 244, 'val': 153}
{'train': <torch.utils.data.dataloader.DataLoader object at 0x7e1c9c4816c0>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7e1c9c482890>}


#***Without writing our own custom class***

In [ ]:
train_dataset_1 = torchvision.datasets.ImageFolder(
    root = '/content/data/hymenoptera_data/train',
    transform = data_transform['train']
)

val_dataset_1 = torchvision.datasets.ImageFolder(
    root = '/content/data/hymenoptera_data/val',
    transform = data_transform['val']
)

batch_size = 4
train_loader_1 = torch.utils.data.DataLoader(
    dataset = train_dataset_1,
    batch_size = batch_size,
    shuffle = True
)

val_loader_1 = torch.utils.data.DataLoader(
    dataset = val_dataset_1,
    batch_size = batch_size,
    shuffle = True
)

In [ ]:
dataloaders_1 = {
    'train' : train_loader_1,
    'val' : val_loader_1
}

dataset_sizes_1 = {
    'train' : len(train_dataset_1),
    'val' : len(val_dataset_1)
}

In [ ]:
print(dataset_sizes_1)
print(dataloaders_1)

{'train': 244, 'val': 153}
{'train': <torch.utils.data.dataloader.DataLoader object at 0x7e1c9c480e80>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7e1c9c481cc0>}


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'epoch :  {epoch + 1}/{num_epochs}')
            print('_' * 40)

            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} loss: {epoch_loss:.4f} | {phase} acc : {epoch_acc:.4f}')

                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training completed in : {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best validation accuracy : {best_acc: 4f}')
        print(f'Best training accuracy : {epoch_acc : 4f}')

        model.load_state_dict(torch.load(best_model_params_path, weights_only = True))
    return model

In [ ]:
model_ft = torchvision.models.resnet18(weights = 'IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
model_ft.fc = torch.nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 95.0MB/s]


In [ ]:
history = train_model(model_ft,
                      criterion,
                      optimizer_ft,
                      exp_lr_scheduler,
                      num_epochs = 25
)

epoch :  1/25
________________________________________
train loss: 0.5317 | train acc : 0.7418
val loss: 0.2296 | val acc : 0.9085

epoch :  2/25
________________________________________
train loss: 0.5575 | train acc : 0.7623
val loss: 0.1283 | val acc : 0.9542

epoch :  3/25
________________________________________
train loss: 0.4902 | train acc : 0.7910
val loss: 0.3055 | val acc : 0.8954

epoch :  4/25
________________________________________
train loss: 0.8311 | train acc : 0.7418
val loss: 0.2588 | val acc : 0.9216

epoch :  5/25
________________________________________
train loss: 0.6513 | train acc : 0.7664
val loss: 0.2758 | val acc : 0.8758

epoch :  6/25
________________________________________
train loss: 0.5750 | train acc : 0.7582
val loss: 0.3762 | val acc : 0.8693

epoch :  7/25
________________________________________
train loss: 0.5532 | train acc : 0.7992
val loss: 0.3213 | val acc : 0.8824

epoch :  8/25
________________________________________
train loss: 0.3769 | 